In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.regularizers import l1,l2

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

In [ ]:
import zipfile

zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
cat_count = len(os.listdir('/content/train/cats'))
dog_count = len(os.listdir('/content/train/dogs'))

counts = [cat_count,dog_count]
print(counts)

plt.pie(counts,startangle = 90) #Perfectly balanced dataset
plt.legend(labels = ['Cats', 'Dogs'])

In [ ]:
#Parameters

img_height = 256
img_width = 256

In [ ]:
train = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (img_height,img_width)
)

In [ ]:
validation = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (img_height,img_width)
)

In [ ]:
model = Sequential()

model.add(layers.Rescaling(1./255, input_shape = (img_height, img_width, 3)))

model.add(layers.Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size = (2,2), strides = 2))

model.add(layers.Conv2D(64, kernel_size = (3,3),  activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size = (2,2), strides = 2))

model.add(layers.Conv2D(128, kernel_size = (3,3), activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size = (2,2), strides = 2))

model.add(layers.Conv2D(256, kernel_size = (3,3), activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size = (2,2), strides = 2))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.25))

model.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
#Compiling the model

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Summary

model.summary()

In [ ]:
hist = model.fit(
    train,
    validation_data = validation,
    epochs = 15
)

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs_range = range(15)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import cv2

def predict_label():
  img = cv2.imread('downloaded_image.jpg')

  img = cv2.resize(img,(256,256))
  img = img.reshape((1, 256, 256, 3))
  label = model.predict(img)

  if label > 0.5:
    print("Dog")
  else:
    print("Cat")
  print(label)

In [ ]:
import requests
import os

def download_image(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image downloaded successfully and saved at {save_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")
    predict_label()

image_url = input("Enter an URL:")
save_path = "downloaded_image.jpg"

download_image(image_url, save_path)